In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm
import pickle
import random

In [2]:
# game data outputted from W1_DataCleaning.ipynb, with stats, goes back to 2010
game_data = pd.read_csv('mydata/womens/matchups.csv')
game_data.head()

,Season,MooreRating_x,MooreRating_y,Seed_x,3ptRate_x,Ast%_x,FT%_x,OppFT%_x,Opp3ptRate_x,OppAst%_x,...,xOffyOffFTRateDiff,AbsxOffyDefAstDiff,AbsyOffxDefAstDiff,xOffyDefAstAvg,yOffxDefAstAvg,xOffyOffAstDiff,TotalPossVarSum,GameScoreVarSum,AvgTotalPoss,MooreNaiveUpsetProbability
0,2010.0,91.81,78.17,4,0.153633,0.562874,0.726562,0.687204,0.318363,0.449704,...,0.103332,0.046362,0.116533,0.539693,0.507970,0.031723,323.329209,249.795260,142.390152,0.111138
1,2010.0,91.81,82.29,4,0.153633,0.562874,0.726562,0.687204,0.318363,0.449704,...,0.137651,0.109423,0.216030,0.617586,0.557719,0.059867,498.758776,240.489802,139.451000,0.192651
2,2010.0,95.45,91.81,1,0.230377,0.550165,0.678383,0.679487,0.302521,0.512938,...,-0.085583,0.100461,0.049937,0.499935,0.537906,-0.037971,356.530636,259.872359,139.373125,0.374739
3,2010.0,92.39,91.81,2,0.226961,0.530850,0.662016,0.665517,0.309621,0.603648,...,-0.118697,0.081146,0.040774,0.490277,0.583261,-0.092984,359.897075,306.882580,141.712500,0.481327
4,2010.0,114.80,91.81,1,0.276465,0.623000,0.723077,0.630556,0.288594,0.486111,...,-0.052871,0.173296,0.076763,0.536352,0.524493,0.011859,372.720173,234.122715,141.959375,0.016799


In [3]:
len(game_data)

630

In [4]:
# game data outputted from W1_DataCleaning.ipynb, no stats, goes back to 2005
game_data_no_stats = pd.read_csv('mydata/womens/matchups_no_stats.csv')
game_data_no_stats.head()

,Season,MooreRating_x,MooreRating_y,Seed_x,Seed_y,Upset,ScoreDiff,SeedDiff,MoorePredictedSpread
0,2005.0,86.08,66.75,5,12,0,22.0,-7,19.33
1,2005.0,86.08,84.71,5,4,0,9.0,1,1.37
2,2005.0,95.16,66.20,2,15,0,21.0,-13,28.96
3,2005.0,95.16,79.34,2,10,0,23.0,-8,15.82
4,2005.0,82.85,79.34,7,10,1,-3.0,-3,3.51


In [5]:
len(game_data_no_stats)

945

In [6]:
X_stats = game_data.drop(columns = ['Upset', 'ScoreDiff'])
y_stats = game_data['Upset']
X_no_stats = game_data_no_stats.drop(columns = ['Upset', 'ScoreDiff'])
y_no_stats = game_data_no_stats['Upset']

In [7]:
X_stats.describe()

,Season,MooreRating_x,MooreRating_y,Seed_x,3ptRate_x,Ast%_x,FT%_x,OppFT%_x,Opp3ptRate_x,OppAst%_x,...,xOffyOffFTRateDiff,AbsxOffyDefAstDiff,AbsyOffxDefAstDiff,xOffyDefAstAvg,yOffxDefAstAvg,xOffyOffAstDiff,TotalPossVarSum,GameScoreVarSum,AvgTotalPoss,MooreNaiveUpsetProbability
count,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,...,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,6.300000e+02
mean,2014.500000,91.447635,77.950905,3.501587,0.269958,0.580175,0.714094,0.685016,0.293521,0.525043,...,0.014510,0.074716,0.076593,0.558668,0.550156,0.008511,327.087600,237.323365,139.227468,1.954093e-01
std,2.874564,9.523172,10.045891,2.517822,0.068446,0.061336,0.041802,0.025582,0.043204,0.051228,...,0.044916,0.053422,0.053848,0.041075,0.039295,0.053630,86.016966,48.006064,4.608569,1.638898e-01
min,2010.000000,71.880000,50.100000,1.000000,0.083661,0.404508,0.549784,0.620482,0.170476,0.399135,...,-0.143509,0.000447,0.000066,0.458758,0.445061,-0.145324,168.263227,116.655992,124.592258,4.646013e-11
25%,2012.000000,84.525000,71.982500,1.000000,0.220174,0.537396,0.682909,0.667307,0.270069,0.486765,...,-0.014806,0.034642,0.033764,0.531133,0.522450,-0.029945,266.695298,202.993680,136.332350,3.303940e-02
50%,2014.500000,89.970000,78.455000,3.000000,0.271959,0.580132,0.713627,0.685484,0.291710,0.516972,...,0.011562,0.065343,0.067098,0.556035,0.548235,0.008747,312.734864,235.222923,139.124415,1.690637e-01
75%,2017.000000,96.700000,84.577500,5.000000,0.316067,0.623000,0.745287,0.700431,0.319613,0.562827,...,0.045697,0.105852,0.110597,0.588055,0.575420,0.045377,368.840148,268.182892,142.071003,3.384913e-01
max,2019.000000,118.220000,107.640000,12.000000,0.487848,0.766323,0.807420,0.745455,0.440853,0.705972,...,0.164782,0.339222,0.343439,0.687049,0.687824,0.189276,975.012208,377.864969,157.054323,4.992431e-01


In [8]:
X_no_stats.describe()

,Season,MooreRating_x,MooreRating_y,Seed_x,Seed_y,SeedDiff,MoorePredictedSpread
count,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000
mean,2012.000000,91.455471,78.216434,3.502646,9.089947,-5.587302,13.239037
std,4.322782,8.848588,9.930082,2.505977,4.323438,4.883500,11.306382
min,2005.000000,71.880000,50.100000,1.000000,1.000000,-15.000000,0.020000
25%,2008.000000,84.850000,72.230000,1.000000,6.000000,-9.000000,4.480000
50%,2012.000000,90.470000,78.610000,3.000000,9.000000,-5.000000,10.350000
75%,2016.000000,96.700000,84.660000,5.000000,13.000000,-1.000000,19.330000
max,2019.000000,118.220000,107.640000,12.000000,16.000000,8.000000,59.880000


In [9]:
# turns the probabilities for all games with a seed diff > 10 to one
def send_probs_to_one(preds, pred_feature_df, pred_indexes):
    seed_diffs = pred_feature_df['SeedDiff'][pred_indexes]
    for i in range(len(seed_diffs)):
        if seed_diffs[i] < -10:
            preds[i, 0] = 1
            preds[i, 1] = 0
        elif seed_diffs[i] > 10:
            preds[i, 0] = 0
            preds[i, 1] = 1
    
    return preds

In [10]:
myCViterator = []
myCViterator_stats = []
for i in [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]:
    trainIndices = X_no_stats[X_no_stats['Season'] != i].index.values.astype(int)
    testIndices = X_no_stats[X_no_stats['Season'] == i].index.values.astype(int)
    myCViterator.append( (trainIndices, testIndices) )
    trainIndices = X_stats[X_stats['Season'] != i].index.values.astype(int)
    testIndices = X_stats[X_stats['Season'] == i].index.values.astype(int)
    myCViterator_stats.append( (trainIndices, testIndices) )

### Logistic Regression

In [20]:
# lasso regularization for feature selection
# coefficients range from 0.02 to ~2000
# each coefficient is 6% larger than last, there are 125 coefficients in total
lasso_reg = [0.02 * (1.02) ** i for i in range(125)]

# ridge regularization to prevent overfitting
ridge_reg = [1e-3, 3e-3, 1e-2, 3e-2, 1e-1, 3e-1, 1e0, 3e0, 1e1, 3e1, 1e2, 3e2, 1e3]

# scale used to scale columns before applying regularization
scale = StandardScaler()

def train_logistic_regression(stats, features = None):
    if features:
        features = features + ['Season']
    
    # if using the stats dataframe or the non stats dataframe
    if stats:
        X = X_stats.copy().drop(columns = ['Season'])
        y = y_stats
        lasso_coef = lasso_reg
        cv_iter = myCViterator_stats
    else:
        X = X_no_stats.copy().drop(columns = ['Season'])
        y = y_no_stats
        lasso_coef = lasso_reg
        cv_iter = myCViterator
        
    features = features.remove('Season')
    
    # list to store results
    feature_list = []
    scores = []
    ridge_coefs = []
    models = []
    num_features = []
    
    # for each possible lasso regularization coefficient
    for c in tqdm(lasso_coef):
        
        if c is not None:
            # dont use season as a feature, use inputted features if given
            if features:
                X_lasso = X[features]
            else:
                X_lasso = X.copy()

            # scale columns before regularization
            X_lasso = pd.DataFrame(scale.fit_transform(X_lasso), columns = X_lasso.columns)

            # fit L1 logistic regression for feature selection
            lasso = LogisticRegression(penalty = 'l1', C = c, random_state = 0, solver = 'saga', max_iter = 10000).fit(X_lasso, y)

            # filter for the columns with nonzero coefficients and the season column
            zero_cols = []
            if features:
                zero_cols = list(set(X.drop(columns = ['Season']).columns) - set(features))
            for i in range(len(lasso.coef_[0])):
                if lasso.coef_[0][i] == 0.0:
                    zero_cols.append(X_lasso.columns[i])
            nonzero_X = X.drop(columns = zero_cols)
        else:
            nonzero_X = X.copy()
        
        # if there is at least 1 nonzero column and the same amount of features hasn't already been built
        if (len(nonzero_X.columns) > 0 and (len(nonzero_X.columns)) not in num_features):
            
                    
            log_model = LogisticRegressionCV(penalty = 'l2', 
                                             Cs = ridge_reg, 
                                             max_iter = 10000, 
                                             random_state = 0, 
                                             solver = "sag",
                                             scoring = 'neg_log_loss',
                                             cv = cv_iter)
            log_fit = log_model.fit(pd.DataFrame(scale.fit_transform(nonzero_X), columns = nonzero_X.columns), y)

            # store model details
            feature_list.append(nonzero_X.columns)
            num_features.append(len(nonzero_X.columns))
            models.append(log_fit)
            ridge_coefs.append(log_fit.C_)
            scores.append(np.mean(log_fit.scores_[1][:, ridge_reg.index(log_fit.C_)]))
                
    # return dataframe of results
    return pd.DataFrame({'Type': ['log' for i in range(len(models))],
                         'Num_Features': num_features,
                         'Features': feature_list,
                         'Model': models,
                         'Model_Coef': ridge_coefs,
                         'Avg_Score': scores,
                         'Stats': [stats for i in range(len(models))]})

### Results

In [21]:
# train the models
#log_results1 = train_logistic_regression(True)
#log_results2 = train_logistic_regression(False)
log_results3 = train_logistic_regression(False, ['SeedDiff', 'MoorePredictedSpread'])
final_results = pd.concat([log_results1, log_results2, log_results3], ignore_index = True)

100%|████████████████████████████████████████████████████████████████████████████████| 125/125 [00:01<00:00, 68.79it/s]


In [22]:
# function to show params of given model by results row index
def params(row_index):
    return final_results.loc[row_index, 'Model_Coef']

In [23]:
# function to show feature importances of given model by results row index
def features(row_index):
    model = final_results.loc[row_index, 'Model']
    model_features = list(final_results.loc[row_index, 'Features'])
    sign_list = []
    coef_list = []
    for i in range(len(model_features)):
        coef = round(model.coef_[0][i], 6)
        coef_list.append(coef)
        if coef > 0:
            sign_list.append('+')
        else:
            sign_list.append('-')

    return_df =  pd.DataFrame({'Feature': model_features,
                         'Sign': sign_list,
                        'Coefficient': coef_list})
    return return_df.reindex(return_df['Coefficient'].abs().sort_values(ascending = False).index)

In [24]:
# print results
final_results.sort_values(by = ['Avg_Score'], ascending = False).head(60)[['Num_Features', 'Model_Coef', 'Stats', 'Avg_Score']]

,Num_Features,Model_Coef,Stats,Avg_Score
17,19,[0.3],True,-0.312267
18,20,[0.3],True,-0.313375
19,21,[0.3],True,-0.313688
15,17,[0.3],True,-0.316552
11,12,[3.0],True,-0.316622
9,10,[10.0],True,-0.316983
14,16,[0.3],True,-0.317544
10,11,[10.0],True,-0.317678
16,18,[0.1],True,-0.317687
20,23,[0.3],True,-0.318304


In [25]:
features(37)

,Feature,Sign,Coefficient
2,MoorePredictedSpread,-,-3.356290
1,SeedDiff,-,-0.616226
0,MooreRating_x,-,-0.032635


In [26]:
final_results.loc[37, 'Features']

Index(['MooreRating_x', 'SeedDiff', 'MoorePredictedSpread'], dtype='object')

In [27]:
pickle.dump(final_results.loc[37, 'Model'], open('models/WProb.sav', 'wb'))